# Projet python

In [2]:
import pandas as pd
from Scrapper import Scrapper

Corrélation avec des facteurs environnementaux
Météo et précipitations : Croise les données d'incendies avec les données météorologiques (températures élevées, sécheresse, précipitations faibles) pour voir s’il existe une corrélation entre les conditions climatiques et les départs de feu. Par exemple, une sécheresse prolongée pourrait être liée à une augmentation du nombre d'incendies.

In [1]:
start_date = "01/10/2023"
end_date = "31/12/2023"
fire_departure_df = Scrapper().get_fire_departure_in_france_data(start_date, end_date)

In [3]:
natural_fire_departure_df = fire_departure_df[fire_departure_df['Nature'] != "Malveillance"]
natural_fire_departure_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271 entries, 0 to 332
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0                           271 non-null    object
 1   Année                   271 non-null    object
 2   Alerte                  271 non-null    object
 3   Département             271 non-null    object
 4   Commune                 271 non-null    object
 5   Surface (ha)            271 non-null    object
 6   Nature                  271 non-null    object
 7   Précision de la donnée  271 non-null    object
dtypes: object(8)
memory usage: 19.1+ KB


In [4]:
natural_fire_departure_df.head()

,,Année,Alerte,Département,Commune,Surface (ha),Nature,Précision de la donnée
0,,2023,01/10/2023 00:17,13,Saint-Chamas,0.0010,-,-
2,,2023,01/10/2023 14:53,34,Nébian,0.4046,Involontaire (particulier),-
3,,2023,02/10/2023 05:25,13,Eyguières,0.0010,-,-
4,,2023,02/10/2023 07:47,13,Eygalières,0.0010,-,-
5,,2023,02/10/2023 08:43,2B,Linguizzetta,0.0010,-,-
